# Challenge 3

In this challenge we will work on the `Orders` data set. In your work you will apply the thinking process and workflow we showed you in Challenge 2.

You are serving as a Business Intelligence Analyst at the headquarter of an international fashion goods chain store. Your boss today asked you to do two things for her:

**First, identify two groups of customers from the data set.** The first group is **VIP Customers** whose **aggregated expenses** at your global chain stores are **above the 95th percentile** (aka. 0.95 quantile). The second group is **Preferred Customers** whose **aggregated expenses** are **between the 75th and 95th percentile**.

**Second, identify which country has the most of your VIP customers, and which country has the most of your VIP+Preferred Customers combined.**

## Q1: How to identify VIP & Preferred Customers?

We start by importing all the required libraries:

In [1]:
# import required libraries
import numpy as np
import pandas as pd

Next, extract and import `Orders` dataset into a dataframe variable called `orders`. Print the head of `orders` to overview the data:

In [2]:
orders = pd.read_csv('Orders.csv')  # Load the dataset into a DataFrame called 'orders'

print(orders.head()) # Display the first 5 rows to understand the structure of the data


   Unnamed: 0  InvoiceNo StockCode  year  month  day  hour  \
0           0     536365    85123A  2010     12    3     8   
1           1     536365     71053  2010     12    3     8   
2           2     536365    84406B  2010     12    3     8   
3           3     536365    84029G  2010     12    3     8   
4           4     536365    84029E  2010     12    3     8   

                           Description  Quantity          InvoiceDate  \
0   white hanging heart t-light holder         6  2010-12-01 08:26:00   
1                  white metal lantern         6  2010-12-01 08:26:00   
2       cream cupid hearts coat hanger         8  2010-12-01 08:26:00   
3  knitted union flag hot water bottle         6  2010-12-01 08:26:00   
4       red woolly hottie white heart.         6  2010-12-01 08:26:00   

   UnitPrice  CustomerID         Country  amount_spent  
0       2.55       17850  United Kingdom         15.30  
1       3.39       17850  United Kingdom         20.34  
2       2.75     

---

"Identify VIP and Preferred Customers" is the non-technical goal of your boss. You need to translate that goal into technical languages that data analysts use:

## How to label customers whose aggregated `amount_spent` is in a given quantile range?


We break down the main problem into several sub problems:

#### Sub Problem 1: How to aggregate the  `amount_spent` for unique customers?

#### Sub Problem 2: How to select customers whose aggregated `amount_spent` is in a given quantile range?

#### Sub Problem 3: How to label selected customers as "VIP" or "Preferred"?

*Note: If you want to break down the main problem in a different way, please feel free to revise the sub problems above.*

Now in the workspace below, tackle each of the sub problems using the iterative problem solving workflow. Insert cells as necessary to write your codes and explain your steps.

In [17]:
# Sub Problem 1:
customer_totals = orders.groupby(['CustomerID', 'Country'])['amount_spent'].sum().reset_index() # Group by CustomerID and Country, and sum the 'amount_spent' for each customer, then reset_index() transforms it into a clean DataFrame with proper columns.

customer_totals.columns = ['CustomerID', 'Country', 'TotalSpent'] # Rename the new column for clarity

print(customer_totals.head()) # Display the first few rows


   CustomerID         Country  TotalSpent
0       12346  United Kingdom    77183.60
1       12347         Iceland     4310.00
2       12348         Finland     1797.24
3       12349           Italy     1757.55
4       12350          Norway      334.40


In [18]:
# Sub Problem 2:
# Calculate the 75th and 95th percentiles of total customer spending:
q75 = customer_totals['TotalSpent'].quantile(0.75)
q95 = customer_totals['TotalSpent'].quantile(0.95)

# Select VIP customers: spent more than 95th percentile:
vip_customers = customer_totals[customer_totals['TotalSpent'] > q95].copy()

# Select Preferred customers: spent between 75th and 95th percentile
preferred_customers = customer_totals[
    (customer_totals['TotalSpent'] > q75) &
    (customer_totals['TotalSpent'] <= q95)
].copy()


In [19]:
#Sub Problem 3:
# Label VIP and Preferred customers safely
vip_customers.loc[:, 'CustomerType'] = 'VIP'
preferred_customers.loc[:, 'CustomerType'] = 'Preferred'

# Combine into a single DataFrame
labeled_customers = pd.concat([vip_customers, preferred_customers], ignore_index=True)

# Display the result
print(labeled_customers.head())


   CustomerID         Country  TotalSpent CustomerType
0       12346  United Kingdom    77183.60          VIP
1       12357     Switzerland     6207.67          VIP
2       12359          Cyprus     6372.58          VIP
3       12409     Switzerland    11072.67          VIP
4       12415       Australia   124914.53          VIP


Now we'll leave it to you to solve Q2 & Q3, which you can leverage from your solution for Q1:

## Q2: How to identify which country has the most VIP Customers?

In [20]:
# Count the number of VIP customers per country
vip_counts_by_country = vip_customers['Country'].value_counts()

# Display the result
print("VIP Customers per Country:")
print(vip_counts_by_country)

# Get the country with the most VIP customers
top_vip_country = vip_counts_by_country.idxmax()


print("Country with the most VIP Customers:")
print(top_vip_country)


VIP Customers per Country:
Country
United Kingdom     178
Germany             11
France               9
Switzerland          3
Spain                2
EIRE                 2
Portugal             2
Japan                2
Cyprus               1
Norway               1
Denmark              1
Australia            1
Finland              1
Singapore            1
Netherlands          1
Channel Islands      1
Sweden               1
Name: count, dtype: int64
Country with the most VIP Customers:
United Kingdom


## Q3: How to identify which country has the most VIP+Preferred Customers combined?

In [21]:
# Count number of VIP + Preferred customers per country using the labeled_customers DataFrame (Sub Problem 3)
combined_counts = labeled_customers['Country'].value_counts()

# Show full distribution
print("VIP + Preferred Customers per Country:")
print(combined_counts)

# Identify the country with the highest number
top_combined_country = combined_counts.idxmax()

print("Country with the most VIP + Preferred Customers:")
print(top_combined_country)

VIP + Preferred Customers per Country:
Country
United Kingdom     934
Germany             39
France              29
Belgium             11
Switzerland          9
Spain                7
Norway               7
Portugal             7
Italy                5
Finland              5
Australia            4
Japan                4
Channel Islands      4
Cyprus               3
Denmark              3
EIRE                 3
Israel               2
Sweden               2
Singapore            1
Iceland              1
Netherlands          1
Austria              1
Lebanon              1
Poland               1
Greece               1
Canada               1
Malta                1
Name: count, dtype: int64
Country with the most VIP + Preferred Customers:
United Kingdom
